In [60]:
%matplotlib inline
import pandas as pd
import numpy as np
import codecs
import os
import re
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Change to your own filepath below
dat_path = "mimic_iii_1_4/{0}"

In [69]:
def clean_text(x, stopwords):
    return " ".join([y.lower() for y in x.split(" ") if y.lower().strip() not in stopwords])
    
# FUNCTIONS NEEDED FOR ALL MODELS  ============================================

# function slightly modified from https://nlpforhackers.io/topic-modeling/

def sort_dict(x):
    return sorted(x.items(), key=lambda l: l[1], reverse=True)

def reconstruct_abstract(x):
    return " ".join(x)

def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):  # loop through each row of H.  idx = row index.  topic = actual row
        print("\nTopic %d:" % (idx))
        #print([(vectorizer.get_feature_names()[i], topic[i])  # printing out words corresponding to indices found in next line
                        #for i in topic.argsort()[:-top_n - 1:-1]])  # finding indices of top words in topic
            
        print_list = [(vectorizer.get_feature_names()[i], topic[i])  
                        for i in topic.argsort()[:-top_n - 1:-1]]
        for item in print_list:
            print(item)

def return_top_topics(model, vectorizer, top_n=5):
        ret = []
        for idx, topic in enumerate(model.components_):  # loop through each row of H.  idx = row index.  topic = actual row
        
        #print([(vectorizer.get_feature_names()[i], topic[i])  # printing out words corresponding to indices found in next line
                        #for i in topic.argsort()[:-top_n - 1:-1]])  # finding indices of top words in topic
            
            print_list = [vectorizer.get_feature_names()[i].title()  
                        for i in topic.argsort()[:-top_n - 1:-1]]
            ret.append("Topic %d: "%(idx+1)+", ".join(print_list))
        return ret


In [15]:
#pat_dat = pd.read_csv(dat_path.format("PATIENTS.csv.gz"))
note_dat = pd.read_csv(dat_path.format("NOTEEVENTS.csv.gz"))
#prescription_dat = pd.read_csv(dat_path.format("PRESCRIPTIONS.csv.gz"))
#pat_dat
#prescription_dat
#Join on subject_ID, Row_ID

C:\Users\johna\anaconda3\envs\analytics\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
note_dat["col_head"] = note_dat["TEXT"].apply(lambda x : re.find[all("\n\n[A-Z].*?:", x))
#master_cols = list(set(note_dat["col_head"].sum()))

In [47]:
med_array = pd.read_excel("Updated/d1853463 Data Dictionary 20220711.xlsx", sheet_name = "Med Categories", engine = "openpyxl")
med_array["med_search_name"] = med_array["GENERIC_NAME"].apply(lambda x : x.rsplit("mg")[0].strip().rsplit(" - ")[-1].rsplit(" ")[0].lower() if type(x) is str else x)
med_array = med_array[["med_search_name","BETA_BLOCKER_FLG","LIPID_LOWERER_FLG","ANTIHYPERTENSIVE_FLG"]].dropna().drop_duplicates()
med_list = [x for x in med_array["med_search_name"].unique() if len(x) > 3]
len(med_list)


130

In [18]:
note_dat_clean = note_dat[note_dat["col_head"].apply(lambda x : len(x) > 1)].reset_index(drop = True)
note_dat_clean

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,col_head
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,"[\n\nService:, \n\nRADIOLOGIC STUDIES:, \n\nHE..."
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...,"[\n\nDate of Birth:, \n\nService:, \n\nHISTORY..."
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...,"[\n\nService:, \n\nAllergies:, \n\nAttending:,..."
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...,"[\n\nService:, \n\nAllergies:, \n\nAttending:,..."
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...,"[\n\nDate of Birth:, \n\nService:, \n\nAllergi..."
...,...,...,...,...,...,...,...,...,...,...,...,...
291942,2079742,32242,163974.0,2121-11-14,2121-11-14 08:03:00,2121-11-14 08:08:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL#5, CGA 26 1/7 weeks.\n\nCVR: ...","[\n\nCVR:, \n\nFEN:, \n\nID:, \n\nGI:, \n\nDEV..."
291943,2079744,32242,163974.0,2121-11-14,2121-11-14 14:47:00,2121-11-14 15:10:00,Nursing/other,Report,14957.0,NaN,Nursing Progress Note\n\n\nFluids: TF 150cc/k...,"[\n\nFluids:, \n\nResp:, \n\nSepsis:, \n\nPare..."
291944,2077115,31767,105243.0,2141-01-05,2141-01-05 14:24:00,2141-01-05 15:01:00,Nursing/other,Report,20700.0,NaN,NICU Attending Admission Note\nPt is a 34 [**6...,"[\n\nPE:, \n\nA/P:, \n\nFEN:]"
291945,2070656,31097,115637.0,2132-01-20,2132-01-20 14:00:00,2132-01-20 14:13:00,Nursing/other,Report,20634.0,NaN,0700-[**2054**] NPN\n\n\nRESP: Infant is in RA...,"[\n\nRESP:, \n\nFEN:, \n\nDEV:, \n\nPARENTING:..."


In [19]:
parsed = []
column_check = ['\n\nDate of Birth:',
 '\n\nService:',
 '\n\nAllergies:',
 '\n\nAttending:',
 '\n\nMajor Surgical or Invasive Procedure:',
 '\n\nHistory of Present Illness:',
 '\n\nPast Medical History:',
 '\n\nSocial History:',
 '\n\nFamily History:',
 '\n\nPhysical Exam:',
 '\n\nGen:',
 '\n\nPertinent Results:',
 '\n\nU/A:',
 '\n\nIMAGING:',
 '\n\nBrief Hospital Course:',
 '\n\nMedications on Admission:',
 '\n\nDischarge Medications:',
 '\n\nDischarge Disposition:',
 '\n\nDischarge Diagnosis:',
 '\n\nSecondary:',
 '\n\nDischarge Condition:',
 '\n\nDischarge Instructions:',
 '\n\nFollowup Instructions:',
 '\n\nCompleted by:']

i = 0
total_row = note_dat_clean.shape[0]
for row in note_dat_clean.iterrows():
    ind, dat = row
    f = {}
    print(ind)
    for col in column_check:#dat["col_head"]:
        #print(col)
        f[col.strip()] = []
        tmp = dat["TEXT"][dat["TEXT"].find(col)+len(col):]
        tmp = tmp[:tmp.find("\n\n")]
        f[col.strip()].append(tmp.strip())
    i += 1
    print("%.2f%%"%((i/total_row)*100))
    parsed.append(pd.DataFrame(f))
    clear_output(wait = True)
    
       
                          

291946
100.00%


In [43]:
def getMedFromList(x, l):
    for item in l:
        if item in x.lower():
            return item
    return np.nan

In [20]:
out_frame = pd.concat(parsed).reset_index(drop = True)

In [82]:
note_dat_fin = pd.concat([note_dat_clean, out_frame], axis = 1)
note_dat_fin = note_dat_fin['ROW_ID	SUBJECT_ID	HADM_ID	CHARTDATE	Physical Exam:	Medications on Admission:	Discharge Medications:'.split("	")].sort_values(by = ["SUBJECT_ID","CHARTDATE"])
note_dat_fin["discharge_med"] = note_dat_fin["Discharge Medications:"].apply(lambda x : getMedFromList(x, med_list))
note_dat_fin = note_dat_fin.dropna(subset = ["discharge_med"]).reset_index(drop = True)
note_dat_fin

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,Physical Exam:,Medications on Admission:,Discharge Medications:,discharge_med
0,30120,11,194540.0,2178-05-11,ON ADMISSION:,none,1. Diabetic.com Starter Kit Kit Sig: One (...,omega-3
1,51783,17,161087.0,2135-05-13,pt. in distress\nSBP 70- 80's\nlungs CTA\ntach...,ASA 325 mg daily,1. Docusate Sodium 100 mg Capsule Sig: One (1)...,metoprolol
2,7824,18,188822.0,2167-10-04,VS: Tm 99.8 HR 87-133 (98) BP 117-166/75-97 RR...,Nortriptylline 50mg qd (started a week PTA)\nS...,1. Nortriptyline 50 mg Capsule Sig: One (1) Ca...,lisinopril
3,25231,19,109235.0,2108-08-11,VS upon arrival to trauma bay:\nT 96.4 BP 127...,Aricept\nToprol\nTrazadone,1. Trazodone 50 mg Tablet Sig: 0.5 Tablet PO H...,metoprolol
4,7237,21,109451.0,2134-09-24,"PHYSICAL EXAMINATION:\nVS: T 97.2, HR 75, BP 1...",. Nifedipine XL 60 mg daily\n2. Calcitriol 0.2...,1. Atorvastatin 80 mg Tablet Sig: One (1) Tabl...,atorvastatin
...,...,...,...,...,...,...,...,...
20383,40723,99966,167228.0,2191-08-25,Admission\nVS: T 96.3 BP 150/57 HR 60 (paced) ...,ATENOLOL - (Prescribed by Other Provider) - Do...,1. atorvastatin 20 mg Tablet Sig: One (1) Tabl...,atorvastatin
20384,38357,99982,151454.0,2156-12-08,Pulse 80\nB/P 85/57 cuff 95/60 by Aline\nO2sat...,Losartan 25 mg daily\n2. Metoprolol succinate ...,1. oxycodone-acetaminophen 5-325 mg Tablet Sig...,metoprolol
20385,40610,99983,117390.0,2193-04-29,Tmax: 35.9 ??????C (96.6 ??????F)\nTcurrent: 3...,- flomax 0.4 mg PO qD\n- Multivitamin,"1. tamsulosin 0.4 mg Capsule, Ext Release 24 h...",metoprolol
20386,9682,99991,151118.0,2185-01-05,"Pre-Admission Examination:\nOn physical exam, ...",1. Labetalol 200 mg Tablet Sig: Two (2) Tablet...,1. Labetalol 200 mg Tablet Sig: Two (2) Tablet...,lisinopril


In [74]:
with open("clinical_stopwords.txt", "r") as fp:
    stopwords = [x.strip() for x in fp.readlines()]
    
#with open("clinical_stopwords.txt", "r") as fp:
#    onix = fp.readlines()
    
docs = [clean_text(x.replace("\n"," ").strip(), stopwords) for x in note_dat_fin["Physical Exam:"]]
#docs = [clean_text(x, onix) for x in docs]

vectorizer = CountVectorizer()#CountVectorizer(max_df=0.4, min_df=3, lowercase=False, max_features=int(len(docs)/2))
doc_term_matrix = vectorizer.fit_transform(docs)

In [75]:
num_topics = 15

t1 = time.time()
nmf_model = NMF(n_components=num_topics, random_state = 0)
nmf_model.fit_transform(doc_term_matrix)
t2 = time.time()

print(f"  Model time: {t2-t1}")

# print out topics

print_topics(nmf_model, vectorizer, top_n=5)

  Model time: 5.1798882484436035

Topic 0:
('abd', 6.266133828636753)
('gen', 5.948513561378813)
('ext', 5.635029498968945)
('cv', 4.70104774535684)
('soft', 4.407979262178844)

Topic 1:
('non', 4.2041614869255035)
('clear', 3.4998494955745145)
('cv', 2.3311346450392234)
('distended', 2.29731107794004)
('tender', 2.2833980523361705)

Topic 2:
('femoral', 3.2954034541562534)
('bruits', 2.9209855945368957)
('dp', 2.865491814342046)
('carotid', 2.8266595534339327)
('chest', 2.242468351647915)

Topic 3:
('left', 9.611412549003006)
('right', 8.314655279523926)
('dp', 1.564098388680098)
('non', 1.2840244955598725)
('radial', 1.0249030247608708)

Topic 4:
('bilaterally', 5.797506234206642)
('normal', 3.1263957844721313)
('intact', 2.940870103866614)
('pupils', 2.281944947652447)
('oriented', 1.9338567111214513)

Topic 5:
('noted', 7.395347371732637)
('bilaterally', 3.423639809254285)
('lesions', 2.779699987747745)
('nc', 1.770274424156982)
('at', 1.5667221869759747)

Topic 6:
('bp', 8.2487725

In [ ]:
#topics 6, 9, 13
#topics 2, 5, 8, 11, 14

In [96]:
doc_topic = nmf_model.transform(doc_term_matrix)
topic_frame = pd.DataFrame(doc_topic)[[6, 9 , 13]]
topic_frame["best_neutral_topic_score"] = topic_frame[[6, 9 , 13]].max(axis = 1)
note_dat_en = pd.concat([note_dat_fin, topic_frame[["best_neutral_topic_score"]]], axis = 1).rename(columns = {"discharge_med":"med_search_name"})
note_dat_en = note_dat_en['med_search_name	best_neutral_topic_score'.split("	")].groupby("med_search_name").median().reset_index()
note_dat_en = note_dat_en.merge(med_array, on = "med_search_name", how = "left")

note_dat_en["BETA_BLOCKER_FLG"] = note_dat_en["BETA_BLOCKER_FLG"] * note_dat_en["best_neutral_topic_score"]
note_dat_en["LIPID_LOWERER_FLG"] = note_dat_en["LIPID_LOWERER_FLG"] * note_dat_en["best_neutral_topic_score"]
note_dat_en["ANTIHYPERTENSIVE_FLG"] = note_dat_en["ANTIHYPERTENSIVE_FLG"] * note_dat_en["best_neutral_topic_score"]

note_dat_en = note_dat_en.applymap(lambda x : np.nan if x == 0 else x)
note_dat_en = note_dat_en["BETA_BLOCKER_FLG	LIPID_LOWERER_FLG	ANTIHYPERTENSIVE_FLG".split(	)].mean()*10
pd.DataFrame(note_dat_en).to_csv("flag_weights.csv")

In [126]:
re.findall("\n\n[A-Z].*?:", note_dat.iloc[120]["TEXT"])

['\n\nDate of Birth:',
 '\n\nService:',
 '\n\nAllergies:',
 '\n\nAttending:',
 '\n\nMajor Surgical or Invasive Procedure:',
 '\n\nHistory of Present Illness:',
 '\n\nPast Medical History:',
 '\n\nSocial History:',
 '\n\nFamily History:',
 '\n\nPhysical Exam:',
 '\n\nGen:',
 '\n\nPertinent Results:',
 '\n\nU/A:',
 '\n\nIMAGING:',
 '\n\nBrief Hospital Course:',
 '\n\nMedications on Admission:',
 '\n\nDischarge Medications:',
 '\n\nDischarge Disposition:',
 '\n\nDischarge Diagnosis:',
 '\n\nSecondary:',
 '\n\nDischarge Condition:',
 '\n\nDischarge Instructions:',
 '\n\nFollowup Instructions:',
 '\n\nCompleted by:']

In [80]:
print(note_dat.iloc[120]["TEXT"])

Admission Date:  [**2189-9-7**]              Discharge Date:   [**2189-9-9**]

Date of Birth:  [**2129-3-28**]             Sex:   F

Service: MEDICINE

Allergies:
Lomotil

Attending:[**Last Name (NamePattern1) 495**]
Chief Complaint:
Pyelonephritis

Major Surgical or Invasive Procedure:
none


History of Present Illness:
Ms. [**Known lastname 496**] is a 60 yo F with hx of chronic hepatitis C who
presents to the ED with dysuria x 4 days, chills, nausea and
vomiting. She first noted hematuria on Saturday, then developed
dysuria, urinary frequency, urgency and incontinence. +anorexia
x 3 days and N/V on the day prior to admission. Denies back pain
but did have mild abdominal discomfort. She did not take her
temperature at home. She has no recent hospitalizations except
for an ED visit for Bell's palsy which was felt to be related to
Lyme disease. She denies ever having a urinary tract infection.
.
In the ED her initial vitals were: T 99.0, HR 102, BP 103/59, RR
16, O2 98% on RA, however 

In [48]:
for x in note_dat.iloc[120]["TEXT"].rsplit("     "):
    tmp = x.rsplit("\n\n")
    colname = [x.strip().rsplit(":")[0] for x in tmp][0]
    print("-----------------------------")
    print(colname)
    #print(tmp)
    for t in tmp:
        print(len(t.rsplit(":")))
        print(t+"\n")
    print("-----------------------------")
    print("\n")
    

-----------------------------
Admission Date
2
Admission Date:  [**2189-9-7**]    

-----------------------------


-----------------------------

1


-----------------------------


-----------------------------
Discharge Date
2
Discharge Date:   [**2189-9-9**]

2
Date of Birth:  [**2129-3-28**]   

-----------------------------


-----------------------------

1


-----------------------------


-----------------------------
Sex
2
Sex:   F

2
Service: MEDICINE

2
Allergies:
Lomotil

3
Attending:[**Last Name (NamePattern1) 495**]
Chief Complaint:
Pyelonephritis

2
Major Surgical or Invasive Procedure:
none


4
History of Present Illness:
Ms. [**Known lastname 496**] is a 60 yo F with hx of chronic hepatitis C who
presents to the ED with dysuria x 4 days, chills, nausea and
vomiting. She first noted hematuria on Saturday, then developed
dysuria, urinary frequency, urgency and incontinence. +anorexia
x 3 days and N/V on the day prior to admission. Denies back pain
but did have mild abdo

In [42]:
for x in note_dat.iloc[120]["TEXT"].rsplit("     "):
    print(x.strip())

Admission Date:  [**2189-9-7**]

Discharge Date:   [**2189-9-9**]

Date of Birth:  [**2129-3-28**]

Sex:   F

Service: MEDICINE

Allergies:
Lomotil

Attending:[**Last Name (NamePattern1) 495**]
Chief Complaint:
Pyelonephritis

Major Surgical or Invasive Procedure:
none


History of Present Illness:
Ms. [**Known lastname 496**] is a 60 yo F with hx of chronic hepatitis C who
presents to the ED with dysuria x 4 days, chills, nausea and
vomiting. She first noted hematuria on Saturday, then developed
dysuria, urinary frequency, urgency and incontinence. +anorexia
x 3 days and N/V on the day prior to admission. Denies back pain
but did have mild abdominal discomfort. She did not take her
temperature at home. She has no recent hospitalizations except
for an ED visit for Bell's palsy which was felt to be related to
Lyme disease. She denies ever having a urinary tract infection.
.
In the ED her initial vitals were: T 99.0, HR 102, BP 103/59, RR
16, O2 98% on RA, however shortly afterwards deve

In [40]:
len("              ")

14

In [41]:
len("             ")

13